# Load Libraries

In [55]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

 # Define Transformations

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
])

# Load the Data

In [57]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

train_data = datasets.CIFAR10( root="./data", train=True, download=True, transform = transform)
test_data = datasets.CIFAR10( root="./data", train=False, download=True, transform = transform)

In [58]:
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=False)

# Build the Architecture

In [59]:
class cnn_cifar(nn.Module):
    def __init__(self):
        super(cnn_cifar, self).__init__()
        self.cnn1 = nn.Conv2d(3, 6, 5)
        self.cnn2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(5*5*16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.cnn1(x)
        x = torch.relu(x)
        x = torch.max_pool2d(x, 2)

        x = self.cnn2(x)
        x = torch.relu(x)
        x = torch.max_pool2d(x, 2)

        x = x.flatten(1)  
        x = self.fc1(x)
        x = torch.relu(x)

        x = self.fc2(x)
        x = torch.relu(x)

        x = self.fc3(x)
        return x

In [60]:
model = cnn_cifar()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Traning

In [62]:
for epoch in range(10):
    for images, labels in (train_loader):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/10], Loss: {loss.item():.4f}")


Epoch [1/10], Loss: 0.7503
Epoch [2/10], Loss: 0.4519
Epoch [3/10], Loss: 0.8774
Epoch [4/10], Loss: 0.8195
Epoch [5/10], Loss: 1.0099
Epoch [6/10], Loss: 0.3275
Epoch [7/10], Loss: 0.2895
Epoch [8/10], Loss: 0.8611
Epoch [9/10], Loss: 1.2610
Epoch [10/10], Loss: 0.6228


# Evaluation

In [76]:
model.eval()
correct = 0
total= 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, pred = torch.max(outputs, 1)
        correct += (pred == labels).sum().item()
        total += labels.size(0)
print(f"Test Accuracy: {100 * correct / total:.2f}%")


Test Accuracy: 64.31%
